In [ ]:
#!pip install gurobipy

In [1]:
import numpy as np
import gurobipy as gp

## Problem 1

In [2]:
obj = np.array([100,200,150,250,225])
lb = np.ones(5)*4
jobMod = gp.Model()
jobx = jobMod.addMVar(5,lb=lb)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-20


In [3]:
# not twice the hours...need all combinations of job comparisons! for each job compare to all other jobs
twiceCons = jobMod.addConstrs(jobx[i] <= 2*jobx[j] for i in range(5) for j in range(5) if i!=j)
# 40 hours per week
fortyCon = jobMod.addConstr(gp.quicksum(jobx[i] for i in range(5)) <= 40)
# jobs 1 and 3 constraint
onethreeCon = jobMod.addConstr(jobx[0]+jobx[2] >= 15)
# jobs 4 and 5 constraint
fourfiveCon = jobMod.addConstr(jobx[3]+jobx[4] <= 15)

In [4]:
jobMod.setObjective(gp.quicksum(obj[i]*jobx[i] for i in range(5)),sense=gp.GRB.MAXIMIZE)

In [5]:
jobMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
jobMod.optimize()

In [6]:
jobx.x

array([ 5., 10., 10., 10.,  5.])

In [7]:
jobMod.objval



7625.0

In [8]:
150*40
# the company makes $7625 per week from your work, so you are profitable

6000

In [9]:
fortyCon.Pi
# if they increase your hours to 41 they'll make $200 more dollars
# but you would have to be paid $225 for that hour
# they shouldn't ask you to work more!

array(200.)

In [38]:
m = gp.Model("task_allocation_with_bonus")

# Create variables
x = m.addVars(5, lb=4, vtype=gp.GRB.CONTINUOUS, name="task")
s = m.addVar(lb=0, vtype=gp.GRB.CONTINUOUS, name="slack")

# Set the objective to maximize the revenue minus your bonus payment
revenue_per_hour = [100, 200, 150, 250, 225]
m.setObjective(sum(revenue_per_hour[i] * x[i] for i in range(5)) - 100 * s, sense = gp.GRB.MAXIMIZE)

# Add constraints
# 1. Each task must receive at least 4 hours. This is already handled by the variable bounds.
# 2. No one task can occupy 2 times the amount of time of any other task.
for i in range(5):
    for j in range(5):
        if i != j:
            m.addConstr(x[i] <= 2 * x[j], f"task_{i+1}_2times_task_{j+1}")

# 3. The total combined time spent on tasks 1 and 3 must be at least 12 hours.
m.addConstr(x[0] + x[2] == 12 + s, "task_1_and_3")

# 4. The total combined time spent on task 4 and 5 cannot exceed 15 hours and is reduced by the slack.
m.addConstr(x[3] + x[4] <= 15 - s, "task_4_and_5")

# 5. You work 40 hours per week.
m.addConstr(sum(x[i] for i in range(5)) <= 40, "total_hours")

# Optimize model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 23 rows, 6 columns and 51 nonzeros
Model fingerprint: 0x041d2cb6
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [4e+00, 4e+00]
  RHS range        [1e+01, 4e+01]
Presolve time: 0.00s
Presolved: 23 rows, 6 columns, 51 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.2500000e+32   9.000000e+30   9.250000e+02      0s
       6    7.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.500000000e+03


In [39]:
m.x

[6.0, 12.0, 6.0, 9.0, 6.0, 0.0]

In [40]:
m.objVal

7500.0

In [24]:
150*40

6000

In [25]:
w.Pi

200.0

## Problem 2

In [29]:
obj2 = np.array([100,200,150,250,225,-100])
lb2 = np.array([4]*5+[0])
jobMod2 = gp.Model()
jobx2 = jobMod2.addMVar(6,lb=lb2)


In [30]:
# not twice the hours...need all combinations of job comparisons! for each job compare to all other jobs
twiceCons = jobMod2.addConstrs(jobx2[i] <= 2*jobx2[j] for i in range(5) for j in range(5) if i!=j)
# 40 hours per week
fortyCon = jobMod2.addConstr(gp.quicksum(jobx2[i] for i in range(5)) <= 40)
# jobs 1 and 3 constraint
onethreeCon = jobMod2.addConstr(jobx2[0]+jobx2[2]-jobx2[5] == 12)
# jobs 4 and 5 constraint
fourfiveCon = jobMod2.addConstr(jobx2[3]+jobx2[4] <= 15-jobx2[5])


In [31]:
jobMod2.setObjective(gp.quicksum(obj2[i]*jobx2[i] for i in range(6)),sense=gp.GRB.MAXIMIZE)

In [32]:

jobMod2.Params.OutputFlag = 0 # tell gurobi to shut up!!
jobMod2.optimize()

In [33]:
jobx2.x

array([ 6., 12.,  6.,  9.,  6.,  0.])

In [37]:
jobMod2.objVal

7500.0

## Problem 3

In [41]:
carMod = gp.Model()
engine = carMod.addMVar(3,vtype='B')
turbo = carMod.addMVar(2,vtype='B')
breaks = carMod.addMVar(2,vtype='B')
interior = carMod.addMVar(2,vtype='B')

In [42]:
carMod.setObjective(50*engine[0]+55*engine[1]+65*engine[2]+5*(turbo[0]+turbo[1])+5*breaks[0]+7*breaks[1]+15*interior[0]+20*interior[1])

In [43]:
engineCon = carMod.addConstr(gp.quicksum(engine[i] for i in range(3))==1)
t1Con = carMod.addConstr(turbo[0] <= engine[0])
t2Con = carMod.addConstr(turbo[1] <= engine[1])
t3con = carMod.addConstr(turbo[0]+turbo[1] <= 1)
breaksCon = carMod.addConstr(breaks[0]+breaks[1]==1)
breaks2Con = carMod.addConstr(2*breaks[1] >= turbo[0] + turbo[1]) # probably don't need the 2, but let's be safe..
interiorCon = carMod.addConstr(interior[0]+interior[1] == 1)
interior2Con = carMod.addConstr(interior[0] <= engine[2]+turbo[0]+turbo[1])
hpCon = carMod.addConstr(450*engine[0]+600*engine[1]+750*engine[2]+200*turbo[0]+150*turbo[1] >= 600)

In [44]:
carMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
carMod.optimize()

In [45]:
engine.x

array([1., 0., 0.])

In [46]:
turbo.x

array([1., 0.])

In [47]:
breaks.x

array([0., 1.])

In [48]:
interior.x

array([1., 0.])

In [49]:
carMod.objval

77.0

In [58]:
m = gp.Model("ferrari_design")

# Create binary variables
e = m.addMVar(3, vtype=gp.GRB.BINARY, name="engine")
t = m.addMVar(2, vtype=gp.GRB.BINARY, name="turbo")
b = m.addMVar(2, vtype=gp.GRB.BINARY, name="breaks")
i = m.addMVar(2, vtype=gp.GRB.BINARY, name="interior")

# Set the objective to minimize the total cost
m.setObjective(50000*e[0] + 55000*e[1] + 65000*e[2] + 
               5000*t[0] + 5000*t[1] + 
               5000*b[0] + 7000*b[1] + 
               15000*i[0] + 20000*i[1], sense = gp.GRB.MINIMIZE)

# Add constraints
# a) You must pick only 1 engine
m.addConstr(sum(e[j] for j in range(3)) == 1, "choose_one_engine")

# b) Turbo1 can only be used on engine1
m.addConstr(t[0] <= e[0], "turbo1_on_engine1")

# c) Turbo2 can only be used on engine2
m.addConstr(t[1] <= e[1], "turbo2_on_engine2")

# d) Only one of the turbos can be used
m.addConstr(sum(t[j] for j in range(2)) <= 1, "one_turbo")

# e) You must pick only 1 set of breaks
m.addConstr(sum(b[j] for j in range(2)) == 1, "choose_one_break")


# f) Breaks2 must be used if you use a turbo
m.addConstr(sum(t[j] for j in range(2)) <= b[1], "break2_if_turbo")

# g) You must pick only 1 interior
m.addConstr(sum(i[j] for j in range(2)) == 1, "choose_one_interior")

# h) Interior 1 can only be used if you use engine 3 or a turbo
m.addConstr(i[0] <= e[2] + sum(t[j] for j in range(2)), "interior1_with_engine3_or_turbo")

# i) The car must have at least 600 hp
m.addConstr(450*e[0] + 200*t[0] + 600*e[1] + 150*t[1] + 750*e[2] >= 600, "min_horsepower")

# Optimize model
m.Params.OutputFlag = 0
m.optimize()



In [59]:
m.objVal

77000.0

In [60]:
e.x

array([1., 0., 0.])

In [61]:
t.x

array([1., 0.])

In [62]:
b.x

array([0., 1.])

In [63]:
i.x

array([1., 0.])